# Sweep Training

We can perform hyperparameter sweep directly on Colab.



```
# This is formatted as code
```

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/brax/blob/main/notebooks/braxlines/experiment_sweep.ipynb)

In [ ]:
#@title Colab setup and imports
#@markdown ## ⚠️ PLEASE NOTE:
#@markdown This colab runs best using a TPU runtime.  From the Colab menu, choose Runtime > Change Runtime Type, then select **'TPU'** in the dropdown.

#@markdown See [config_utils.py](https://github.com/google/brax/blob/main/brax/experimental/braxlines/common/config_utils.py)
#@markdown for the configuration format.
from datetime import datetime
import importlib
import os
import pprint

from IPython.display import HTML, clear_output

try:
  import brax
except ImportError:
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

agent_module = 'brax.experimental.braxlines.vgcrl.train' #@param{'type': 'string'}
output_path = '' #@param{'type': 'string'}
start_count = 0 # @param{'type': 'integer'}
end_count = 100000000 # @param{'type': 'integer'}
experiment_path = None #@param{'type': 'raw'}
experiment_path=experiment_path or datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = f'{output_path}/{experiment_path}'

from brax.experimental.braxlines.common import config_utils
train = importlib.import_module(agent_module)
if "COLAB_TPU_ADDR" in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

config = [
  dict(
      # env_name = ['ant', 'halfcheetah', 'uni_ant', 'bi_ant'],
      env_name = 'ant',
      env_space = 'vel',
      # algo_name = ['gcrl', 'cdiayn', 'diayn', 'diayn_full'],
      algo_name = ['gcrl', 'diayn'],
      seed = [0],
      normalize_obs_for_disc = [False],
      spectral_norm = [True],
      ppo_params = dict(
        num_timesteps=int(2e8),
        reward_scaling=10,
        episode_length=1000,
        normalize_observations=True,
        action_repeat=1,
        unroll_length=[5, 10, 20],
        num_minibatches=32,
        num_update_epochs=4,
        discounting=0.95,
        learning_rate=[3e-4, 3e-5],
        entropy_cost=1e-2,
        num_envs=[2048, 4096],
        batch_size=1024,)
  ),
]

prefix_keys = config_utils.list_keys_to_expand(config)
for c, p in zip(config, prefix_keys):
  c.update(dict(prefix_keys=p))
config_count= config_utils.count_configuration(config)
start_count= max(start_count, 0)
end_count = min(end_count, sum(config_count))
print(f'Loaded agent_module={agent_module}')
print(f'Loaded {sum(config_count)}({config_count}) experiment configurations')
print(f'Set start_count={start_count}, end_count={end_count}')
print(f'Set prefix_keys={prefix_keys}')
print(f'Set output_dir={output_path}')

In [ ]:
#@title Launch experiments

return_dict = {}
for i in range(start_count, end_count):
  c= config_utils.index_configuration(config, index=i, count=config_count)
  task_name = config_utils.get_compressed_name_from_keys(
      c, train.TASK_KEYS)
  experiment_name = config_utils.get_compressed_name_from_keys(
      c, c.pop('prefix_keys'))
  output_dir = f'{output_path}/{task_name}/{experiment_name}'
  print(f'[{i+1}/{sum(config_count)}] Starting experiment...')
  print(f'\t config: {pprint.pformat(c, indent=2)}')
  print(f'\t output_dir={output_dir}')
  print(f'\t previous time_to_jit={return_dict.get("time_to_train", None)}')
  print(f'\t previous time_to_train={return_dict.get("time_to_jit", None)}')
  return_dict = {}
  try:
    train.train(c, output_dir=output_dir, return_dict=return_dict)
  except Error as e:
    print(f'[{i+1}/{sum(config_count)}] FAILED experiment {e}')